In [2]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import yfinance as yf

In [3]:
tickers = [
    'AAPL', 'TSLA', 'C' 
]

In [4]:
yf.Ticker('AAPL').history("1y")

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-05-12,78.925690,79.387581,77.207273,77.331436,162301200,0.00,0.0
2020-05-13,77.515191,78.458837,75.295149,76.397720,200622400,0.00,0.0
2020-05-14,75.617982,76.929147,74.877966,76.867065,158929200,0.00,0.0
2020-05-15,74.584939,76.459803,74.550170,76.412621,166348400,0.00,0.0
2020-05-18,77.768498,78.595423,77.060765,78.212997,135178400,0.00,0.0
...,...,...,...,...,...,...,...
2021-05-05,128.980915,130.228796,127.753005,127.882790,84000900,0.00,0.0
2021-05-06,127.673135,129.529982,126.914422,129.520004,78128300,0.00,0.0
2021-05-07,130.850006,131.259995,129.479996,130.210007,78892700,0.22,0.0


In [5]:
import datetime as dt

In [6]:
class Balance(object):
    def __init__(self, money, commision, ticker):
        self.money = money
        self.amount = 0
        self.commision = commision
        self.history = yf.Ticker(ticker).history("5y")
    
    
    def buy(self, amount, price):
        self.money -= amount * price
        self.money -= amount * price * (self.commision)
        self.amount += amount
        
    
    def buy_all_at_open(self, ID):
        self.buy(self.money / (self.history.Open[ID] * (1 + self.commision)), self.history.Open[ID])
        
    
    def sell(self, amount, price):
        self.money += amount * price
        self.money -= amount * price * (self.commision)
        self.amount -= amount
        
        
    def sell_all_at_close(self, ID):
        self.sell(self.amount, self.history.Close[ID])
      
    
    def sell_all_by_price(self, price):
        self.sell(self.amount, price)
        
        
    def calculate_money(self, ID):
        money = self.money
        money += self.history.Close[ID] * self.amount
        return money
        
        
    def make_process(self, t1, t2):
        for ID in range(1, len(self.history)):
            overnight_return = (self.history.Close[ID - 1] - self.history.Open[ID]) / self.history.Close[ID - 1]
            if overnight_return >= t1:
                self.buy_all_at_open(ID)

                high_return = (self.history.High[ID] - self.history.Open[ID]) / self.history.Open[ID]

                if high_return >= t2:
                    self.sell_all_by_price(self.history.Open[ID] * (1 + t2))
                else:
                    self.sell_all_at_close(ID)


                
    def get_len(self):
        return len(self.history) - 1

In [7]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [19]:
threshold = np.linspace(0.05 / 100, 2.5 / 100, 20)
fig3D = make_subplots(
    rows=3, cols=1,
    specs=[[{'type': 'surface'}], [{'type': 'surface'}], [{'type': 'surface'}]],
    subplot_titles=tickers)

for num_ticker in range(3):
    
    ticker = tickers[num_ticker]
    Xt1, Yt2, Zmoney = [], [], []
    mx = 0; bt1 = 0; bt2 = 0
    for t1 in threshold:
        for t2 in threshold:
            bal = Balance(1, 0.025 / 100, ticker)
            
            bal.make_process(t1, t2)
                    
            Xt1.append(t1)
            Yt2.append(t2)
            Zmoney.append(bal.calculate_money(bal.get_len()))
            if (Zmoney[len(Zmoney) - 1] > mx): 
                mx = Zmoney[len(Zmoney) - 1]
                bt1, bt2 = t1, t2
    
    print(f'Best thresholds for {ticker} are {bt1, bt2} and money made {mx}')
    fig3D.add_trace(
        go.Mesh3d(x=Xt1, y=Yt2, z=Zmoney, colorscale='Viridis', showscale=False, name=ticker),
        row=num_ticker+1, col=1
    )
    print(f'Made {num_ticker + 1} of 3 tickers')
    
            
fig3D.update_layout(
        scene = dict(
                    xaxis_title='X for t1 threshold',
                    yaxis_title='Y for t2 threshold',
                    zaxis_title='Z for earned money'
        ),
    title_text='Chart how money depends on t1-t2 thresholds',
    height=3000,
    width=1000
)

fig3D.show()            

Best thresholds for AAPL are (0.0030789473684210526, 0.012105263157894737) and money made 1.7392579376882722
Made 1 of 3 tickers
Best thresholds for TSLA are (0.008236842105263159, 0.02242105263157895) and money made 1.9492812575059695
Made 2 of 3 tickers
Best thresholds for C are (0.009526315789473685, 0.013394736842105263) and money made 1.5184761646446163
Made 3 of 3 tickers


Best thresholds for AAPL are (0.0030789473684210526, 0.012105263157894737) and money made 1.7392579376882722

Best thresholds for TSLA are (0.008236842105263159, 0.02242105263157895) and money made 1.9492812575059695

Best thresholds for C are (0.009526315789473685, 0.013394736842105263) and money made 1.5184761646446163



In [27]:
!pip install psutil

     |████████████████████████████████| 236 kB 1.3 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.8/bin/python3.8 -m pip install --upgrade pip' command.


In [29]:
fig3D.write_html('strategy_t1_t2_with_commision.html')

In [30]:
threshold = np.linspace(0.05 / 100, 2.5 / 100, 20)
fig3D = make_subplots(
    rows=3, cols=1,
    specs=[[{'type': 'surface'}], [{'type': 'surface'}], [{'type': 'surface'}]],
    subplot_titles=tickers)

for num_ticker in range(3):
    
    ticker = tickers[num_ticker]
    Xt1, Yt2, Zmoney = [], [], []
    mx = 0; bt1 = 0; bt2 = 0
    for t1 in threshold:
        for t2 in threshold:
            bal = Balance(1, 0.0 / 100, ticker)
            
            bal.make_process(t1, t2)
                    
            Xt1.append(t1)
            Yt2.append(t2)
            Zmoney.append(bal.calculate_money(bal.get_len()))
            if (Zmoney[len(Zmoney) - 1] > mx): 
                mx = Zmoney[len(Zmoney) - 1]
                bt1, bt2 = t1, t2
    
    print(f'Best thresholds for {ticker} are {bt1, bt2} and money made {mx}')
    fig3D.add_trace(
        go.Mesh3d(x=Xt1, y=Yt2, z=Zmoney, colorscale='Viridis', showscale=False, name=ticker),
        row=num_ticker+1, col=1
    )
    print(f'Made {num_ticker + 1} of 3 tickers')
    
            
fig3D.update_layout(
        scene = dict(
                    xaxis_title='X for t1 threshold',
                    yaxis_title='Y for t2 threshold',
                    zaxis_title='Z for earned money'
        ),
    title_text='Chart how money depends on t1-t2 thresholds',
    height=3000,
    width=1000
)

fig3D.show()            

Best thresholds for AAPL are (0.0030789473684210526, 0.012105263157894737) and money made 2.0652681652141096
Made 1 of 3 tickers
Best thresholds for TSLA are (0.005657894736842105, 0.02242105263157895) and money made 2.2850427553052457
Made 2 of 3 tickers
Best thresholds for C are (0.0017894736842105263, 0.010815789473684211) and money made 1.8083812007857178
Made 3 of 3 tickers


In [31]:
fig3D.write_html('strategy_t1_t2_no_commision.html')